In [1]:
import numpy as np
import matplotlib.pyplot as plt

## Encoder

Define a simple class to encode symbols (states and observations names/strings) as integers (IDs).
Basically, we use two entangled data structures: a dictionary (symbol -> ID) and a list (Id -> symbol).

Then, we can represent the HMM parameters as arrays and matrices indexed by integers (IDs) instead of strings.
But, at the same time, we can keep the mapping between symbol name (string) and its ID (integer).

In [2]:
class Encoder:
    """Encode a set of N symbols (strings) into integer IDs from {0, 1, ..., N-1}

    The encoder can be frozen at some point and then no new symbol can be added.
    When a frozen encoder gets a request for an unknown symbol, it raises an
    exception.
    """

    def __init__(self, symbols: list = None, frozen: bool = False):
        """Create a new encoder with optional list of symbols.

        Args:
            symbols (list[str]): list of symbols (default is None)
            frozen (bool): whether the encoder includes new symbols passed to method `get_id(s)` (default is False)
        """
        self.symbol_to_id: dict[str, int] = {}
        self.id_to_symbol: list[str] = []
        self.frozen: bool = False

        if symbols is not None:
            for s in symbols:
                self.get_id(s)

        if frozen:
            self.frozen = True

    def get_id(self, s: str) -> int:
        """return the ID (integer) corresponding to symbols `s`.

        If symbols `s` has not been encoded already and `self.frozen == False`,
        then include `s` in the mapping, assign a new ID for it, and return it.
        If symbols `s` has not been encoded already and `self.frozen == True`,
        raise an exception.

        Args:
            s (str): symbol to be encoded

        Returns:
            int: ID of the encoded symbol
        """
        if s not in self.symbol_to_id:
            if self.frozen:
                raise ValueError(f"Symbol {s} not in frozen encoder {self}")

            new_id = len(self.id_to_symbol)
            self.symbol_to_id[s] = new_id
            self.id_to_symbol.append(s)
            return new_id

        return self.symbol_to_id[s]

    def get_symbol(self, id: int) -> str:
        """return the symbols associated with `id`

        Args:
            id (int): ID of the symbol

        Returns:
            str: the symbol associated with the given ID
        """
        return self.id_to_symbol[id]

    def __repr__(self):
        return f"Encoder: {self.symbol_to_id}"

In [3]:
# define the set of observations (x_enc) and the set of states (y_enc)
x_enc = Encoder(symbols=["Walk", "Shop", "Clean"], frozen=True)
y_enc = Encoder(symbols=["Sunny", "Rainy"], frozen=True)
print(x_enc)
print(y_enc)

Encoder: {'Walk': 0, 'Shop': 1, 'Clean': 2}
Encoder: {'Sunny': 0, 'Rainy': 1}


## HMM
Define the model parameters:
- $\pi_i$ is the start probability for state $i$: $P(y_1=i)$
- $a_{ij}$ is the transition probabilitiy from state $i$ to state $j$: $P(y_t=j|y_{t-1}=i)$
- $b_j(k)$ is the emission probability of observation $k$ when in state $j$: $P(x_t=k|y_t=j)$

We will use Numpy to represent the parameters (vectors and matrices).
This Python library is very popular and allows us to perform algebra operations very easily.
In this exercise, we are not going to use a lot of the power of Numpy.
But in the next exercises, we will need it more offen.
So, it is now a good time to start learning it.

Remember that in both Python and Numpy the base index for arrays and lists is 0 (*zero*),
  which is different from the math equations we used in the lectures.

In [4]:
pi = np.array([0.6, 0.4])

a = np.array(
    [
        [0.7, 0.3],  # transitions a_0j (from state 0 (Sunny) to some state j)
        [0.4, 0.6],  # transitions a_1j (from state 1 (Rainy) to some state j)
    ]
)

b = np.array(
    [
        [0.6, 0.3, 0.1],  # emissions b_0(k) (in state 0 (Sunny) emits k)
        [0.1, 0.4, 0.5],  # emissions b_1(k) (in state 1 (Rainy) emits k)
    ]
)

## Viterbi Algorithm

We now implement the Viterbi algorithm.
We make all computations in the *log domain* to avoid underflow issues.

### Pseudo-code

Given input sequence $X=(x_1,x_2,\ldots,x_T)$ and HMM parameters $\pi, a, b$, 
  as described above,
  return $Y^* = \arg\max_Y P(Y|X)$

- Initialization
  - for $i=1,\ldots,N$
    - $\log\delta_1(i) = \log\pi_i + \log b_i(x_1)$, 
    - $\psi_1(i) = -1$
- for $t = 2, \ldots, T$
  - for $j = 1, \ldots, N$
    - $\log\delta_t(j) = \max_i (\log\delta_{t-1}(i) + \log a_{ij}) + \log b_j(x_t)$
    - $\psi_t(j) = \arg\max_i (\log\delta_{t-1}(i) + \log a_{ij})$
- Recover best sequence $Y^* = (y^*_1, \ldots, y^*_T) = \arg\max_y P(Y|X)$
  - $y^*_T = \arg\max_i\delta_T(i)$
  - for $t = T-1, \ldots, 1$
   - $y^*_t = \psi_{t+1}(y^*_{t+1})$

### Code

In [5]:
def viterbi(x, pi, a, b):
    """Runs the Viterbi algorithm for input `x` using the HMM given by `pi`, `a` and `b`.

    Args:
        x (list[int]): list of observations (integer IDs)
        pi (np.array): start probabilities.
            It is an array with one value for each state.
        a (np.array): transition probabilities, where `a[i, j]` contains
            the probability for a transition from state `i` to state `j`.
        b (np.array): emission probabilities, where `b[j, k]` contains
            the probability of emiting observation `k` when in state `j`.

    Returns:
        tuple: the list containing the most likely sequence of state for the given `x`,
            the `log_delta` table containing the computed log probabilities,
            and the `psi` table.
    """
    num_states = pi.size
    seq_len = len(x)

    # let's work on the log domain (base 2)
    log_pi = np.log2(pi)
    log_a = np.log2(a)
    log_b = np.log2(b)

    log_delta = np.zeros((seq_len, num_states))
    psi = np.full((seq_len, num_states), -1, dtype=np.int32)

    # initilization
    for i in range(num_states):
        log_delta[0, i] = log_pi[i] + log_b[i, x[0]]

    for t in range(1, seq_len):
        for j in range(num_states):
            # State j is the state in time-step t.
            # We now iterate over all previous states i computing
            # log_delta[t-1, i] + log_a[i, j] and stores the maximum value and
            # also the argmax (the index of the maxium value).
            delta_max = -np.inf  # minus infinity
            delta_argmax = -1  # dummy value
            for i in range(num_states):
                log_delta_a = log_delta[t - 1, i] + log_a[i, j]
                if log_delta_a > delta_max:
                    delta_max = log_delta_a
                    delta_argmax = i

            # log_delta is the max value plus the emission probability on time-step t
            log_delta[t, j] = delta_max + log_b[j, x[t]]

            # psi is the argmax
            psi[t, j] = delta_argmax

    # now we recover the most likely sequence of states backwards, from the last
    # time step (in Python, -1 represents the last index of a list/array) up to
    # the first one.
    y = []
    # here we use the argmax() method of a numpy array to find the best
    # state on the last time step
    y.append(log_delta[-1].argmax())
    for t in range(seq_len - 1, 0, -1):
        y.append(psi[t, y[-1]])

    # the list y is backwards, so reverse it and return
    y.reverse()
    return y, log_delta, psi

In [6]:
# encode an input sequence
x = [x_enc.get_id(s) for s in ["Walk", "Shop", "Clean", "Shop", "Walk"]]

# call Viterbi algorithm
y, log_delta, psi = viterbi(x, pi, a, b)

# display delta and psi
print(f"log_delta:\n{log_delta}\n")
print(f"psi:\n{psi}\n")

# display the resulting sequence of states (encoded and decoded)
print(f"y:\n{y}\n")
print(f"decoded y:\n{[y_enc.get_symbol(i) for i in y]}\n")

log_delta:
[[ -1.47393119  -4.64385619]
 [ -3.72546996  -4.53282488]
 [ -7.56197122  -6.26979047]
 [ -9.32868416  -8.32868416]
 [-10.38757785 -12.38757785]]

psi:
[[-1 -1]
 [ 0  0]
 [ 0  1]
 [ 1  1]
 [ 1  1]]

y:
[0, 1, 1, 1, 0]

decoded y:
['Sunny', 'Rainy', 'Rainy', 'Rainy', 'Sunny']



### Vectorized Viterbi Algorithm

I mentioned above that Numpy is great for linear algebra (basic operations on vectors and matrices).
We can write a much cleaner version of the Viterbi algorithm using Numpy linear algebra functionalities.
Some authors call this a *vectorized version* of the algorithm.
Usually, this means replacing `for` loops by array-array, array-matrix and matrix-matrix operations.

In the following, I present a vectorized version of Viterbi algorithm.

In [8]:
def viterbi_v(x, pi, a, b):
    num_states = pi.size
    seq_len = len(x)

    # let's work on the log domain (base 2)
    log_pi = np.log2(pi)
    log_a = np.log2(a)
    log_b = np.log2(b)

    log_delta = np.zeros((seq_len, num_states))
    psi = np.full((seq_len, num_states), -1, dtype=np.int32)

    # initilization
    log_delta[0] = log_pi + log_b[:, x[0]]

    for t in range(1, seq_len):
        # Sum log_delta[t-1] (array of log_delta in the previous step) with
        # log_a matrix. Since delta[t-1] is an array, we want to broadcast its
        # second dimension through the first dimension of log_a (which
        # represents the i state on a transition i->j). So that, in the end,
        # we have log_delta_a[i,j] = log_delta[t-1,i] + log_a[i,j]
        log_delta_a = log_delta[t - 1].reshape(-1, 1) + log_a

        # log_delta is the max value plus the emission probability on
        # time-step t. The max is taken along the first dimension, i.e., the max
        # along the previous state `i` for each state `j`.
        log_delta[t] = log_delta_a.max(axis=0) + log_b[:, x[t]]

        # psi is the argmax
        psi[t] = log_delta_a.argmax(axis=0)

    # now we recover the most likely sequence of states backwards, from the last
    # time step (in Python, -1 represents the last index of a list/array) up to
    # the first one.
    y = []
    # here we use the argmax() method of a numpy array to find the best
    # state on the last time step
    y.append(log_delta[-1].argmax())
    for t in range(seq_len - 1, 0, -1):
        y.append(psi[t, y[-1]])

    # the list y is backwards, so reverse it and return
    y.reverse()
    return y, log_delta, psi

In [9]:
# call Viterbi algorithm
y_v, log_delta_v, psi_v = viterbi_v(x, pi, a, b)

# display log_delta and psi
print(f"log_delta:\n{log_delta_v}\n")
print(f"psi:\n{psi_v}\n")

# display the resulting sequence of states (encoded and decoded)
print(f"y:\n{y_v}\n")
print(f"decoded y:\n{[y_enc.get_symbol(i) for i in y_v]}\n")

log_delta:
[[ -1.47393119  -4.64385619]
 [ -3.72546996  -4.53282488]
 [ -7.56197122  -6.26979047]
 [ -9.32868416  -8.32868416]
 [-10.38757785 -12.38757785]]

psi:
[[-1 -1]
 [ 0  0]
 [ 0  1]
 [ 1  1]
 [ 1  1]]

y:
[0, 1, 1, 1, 0]

decoded y:
['Sunny', 'Rainy', 'Rainy', 'Rainy', 'Sunny']



When vectorizing for loops it makes sense to test whether the output with respect to a certain input is the same as a known-to-be-correct implementation with for-loops. In this case, since the Viterbi algorithm is deterministic, we can check whether the outputs of our vectorized version match the outputs of the implementation with for-loops. In some cases, e.g. if the order of computations changes, numerical equality might not be realistic. In this case, we can check whether the outputs are close ...

In [10]:
assert (log_delta_v == log_delta).all()
assert y_v == y
assert (psi_v == psi).all()

In [11]:
assert np.allclose(log_delta_v, log_delta)
assert np.allclose(y_v, y)
assert np.allclose(psi_v, psi)